##Preparation

In [1]:
#importing libraries and packages
!pip3 install movieposters
import movieposters as mp
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
#reading the data
url1 = "https://drive.google.com/file/d/1sc_yJw6Ej7hmS36OIQHl1UX6h6Jq32uN/view?usp=share_link"
path1 = 'https://drive.google.com/uc?export=download&id='+url1.split('/')[-2]
movies = pd.read_csv(path1)

url2 = "https://drive.google.com/file/d/1shB74shA6w-rOcHHANDiXkiq4-RO-uyP/view?usp=share_link"
path2 = 'https://drive.google.com/uc?export=download&id='+url2.split('/')[-2]
ratings = pd.read_csv(path2)

url3 = "https://drive.google.com/file/d/1Kwuudazbm-jcRIxGO2g3HipoFFRlU17x/view?usp=share_link"
path3 = 'https://drive.google.com/uc?export=download&id='+url3.split('/')[-2]
links = pd.read_csv(path3)

url4 = "https://drive.google.com/file/d/1hgwga5-UMVYEM3qpQT_3vlLerJ3_F2B9/view?usp=share_link"
path4 = 'https://drive.google.com/uc?export=download&id='+url4.split('/')[-2]
tags = pd.read_csv(path4)

In [11]:
ratings.userId.unique


<bound method Series.unique of 0           1
1           1
2           1
3           1
4           1
         ... 
100831    610
100832    610
100833    610
100834    610
100835    610
Name: userId, Length: 100836, dtype: int64>

#Recommenders

##Popularity based for cold start

In [3]:
genres = movies['genres'].str.split("|", expand=True)
genre_list = pd.unique(genres[[0, 1, 2, 3, 4, 5, 6]].values.ravel('K')) # flatten the array in the order it appears in the memory
fin_genre_list = np.delete(genre_list, np.where(genre_list == '(no genres listed)') | (genre_list == None)) # cleaning
fin_genre_list

array(['Adventure', 'Comedy', 'Action', 'Drama', 'Crime', 'Children',
       'Mystery', 'Animation', 'Documentary', 'Thriller', 'Horror',
       'Fantasy', 'Western', 'Film-Noir', 'Romance', 'Sci-Fi', 'Musical',
       'War', 'IMAX'], dtype=object)

In [7]:
def pop_rec(genre, year, n_output):

    scaler = MinMaxScaler()
    rating_df = pd.DataFrame(ratings.groupby('movieId')['rating'].mean()) # group movies and get their avarage rating
    rating_df['rating_count'] = ratings.groupby('movieId')['rating'].count() # get rating count of each movie(how many times each movie was rated)
    scaled_df = pd.DataFrame(scaler.fit_transform(rating_df), index=rating_df.index, columns=rating_df.columns) # scale the ratings and rating counts
  #scaled_df
    scaled_df["hybrid"] = scaled_df['rating'] + scaled_df['rating_count'] # add up rating and rating count for each mivie
    sort_rate = pd.DataFrame(scaled_df["hybrid"].sort_values(ascending=False))
    pattern = '\((\d{4})\)'
    recommend1 = sort_rate.merge(rating_df.merge(movies, how='left', left_index=True, right_on="movieId"), how='left', left_index=True, right_index=True)
    recommend1['year'] = movies.title.str.extract(pattern, expand=False)
    recommend2 = recommend1.dropna()
    recommender3 = recommend2.loc[recommend2.genres.str.contains(genre)] 
    recommender3['year'] = recommender3['year'].astype(int)
    recommender4 = recommender3.loc[(recommender3["year"] >= year-5) & (recommender3["year"] <= year+5)]
    return pd.DataFrame(recommender4['title']).set_index('title').head(n_output)

In [8]:
pop_rec("Drama", 2001, 10)

C:\Users\Olia\AppData\Local\Temp\ipykernel_16284\1552168700.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommender3['year'] = recommender3['year'].astype(int)


""
title
Billy Elliot (2000)
Last Dance (1996)
Career Girls (1997)
Autumn in New York (2000)
187 (One Eight Seven) (1997)
"I Love You, I Love You Not (1996)"
Spy Game (2001)
In the Company of Men (1997)
Soul Food (1997)


##**Pictures**

In [ ]:
df.index#[0]

Index(['Billy Elliot (2000)', 'Last Dance (1996)', 'Career Girls (1997)',
       'Autumn in New York (2000)', '187 (One Eight Seven) (1997)',
       'I Love You, I Love You Not (1996)', 'Spy Game (2001)',
       'In the Company of Men (1997)', 'Soul Food (1997)', 'Selena (1997)'],
      dtype='object', name='title')

In [ ]:
posters_list = []
for i in df.index:
    link = mp.get_poster(title= i)
    title_postlink = i, link
    posters_list.append(title_postlink)
posters_list  

[('Billy Elliot (2000)',
  'https://m.media-amazon.com/images/M/MV5BNGY3NWYwNzctNWU5Yi00ZjljLTgyNDgtZjNhZjRlNjc0ZTU1XkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_QL75_UX380_CR0,0,380,562_.jpg'),
 ('Last Dance (1996)',
  'https://m.media-amazon.com/images/M/MV5BMjAyN2EwNzAtZjQ4NS00NTQ3LWE1MzctYmIwOTA4Nzc2ZGZhXkEyXkFqcGdeQXVyNjQ2MjQ5NzM@._V1_QL75_UY562_CR1,0,380,562_.jpg'),
 ('Career Girls (1997)',
  'https://m.media-amazon.com/images/M/MV5BNzZiNzNjOGItNzVlZC00MGU0LTgyMDItZGJjZTFiOTZmZDU2XkEyXkFqcGdeQXVyMTAwMzUyOTc@._V1_QL75_UX380_CR0,3,380,562_.jpg'),
 ('Autumn in New York (2000)',
  'https://m.media-amazon.com/images/M/MV5BMWFlM2VmYTUtZmFmMC00N2Y1LWE3NmUtYWI5MWZiMTY3NDRiXkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_QL75_UY562_CR1,0,380,562_.jpg'),
 ('187 (One Eight Seven) (1997)',
  'https://m.media-amazon.com/images/M/MV5BYTc1MTUxNmYtY2I3My00MmNkLWI3NmUtMDc1YWNhMzBiMDQzXkEyXkFqcGdeQXVyNjU0NTI0Nw@@._V1_QL75_UY562_CR1,0,380,562_.jpg'),
 ('I Love You, I Love You Not (1996)',
  'https://m.media-amazon.com/images/M

##Items correlation

In [ ]:
rating_pivot = pd.pivot_table(ratings, values = 'rating', columns = 'movieId', index = 'userId')
rating_pivot

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,2.5,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def recom_m(n, m):
    rating_pivot = pd.pivot_table(ratings, values = 'rating', columns = 'movieId', index = 'userId')
    correl_df = rating_pivot.corrwith(rating_pivot[n]) #correlation between the target item and all other items
    correl_df = pd.DataFrame(correl_df.sort_values(ascending=False)) #convert the Series of the correlation to a dataframe and sort it to have top items with the highest correlations
    correl_df1 = correl_df.merge(pd.DataFrame(movies[["movieId", "title"]]), on="movieId", how="left") #get names of the items
    correl_df1.dropna(inplace=True) #drop nulls
    rating_count = pd.DataFrame(ratings.groupby('movieId')['rating'].mean()) #create a dataframe fore the count of the ratings
    rating_count['rating_count'] = ratings.groupby('movieId')['rating'].count() #get the count of the rating to exclude items that are rated only few times
    rating_count.drop(1, inplace=True) #drop the target item itself from the df
    movie_corr_summary = correl_df1.join(rating_count['rating_count']) #join raitings with rating count
    top10 = movie_corr_summary[movie_corr_summary['rating_count']>=10].sort_values(0, ascending=False) #dropping items that are rated less than 10 times

    return top10.head(m)

In [ ]:
recom_m(10, 10)

/usr/local/lib/python3.9/dist-packages/numpy/lib/function_base.py:2821: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/usr/local/lib/python3.9/dist-packages/numpy/lib/function_base.py:2680: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


,movieId,0,title,rating_count
2,64249,1.0,Shrek the Halls (2007),110.0
52,4210,1.0,Manhunter (1986),28.0
97,104419,1.0,Justice League: Crisis on Two Earths (2010),10.0
95,106438,1.0,Philomena (2013),84.0
94,7255,1.0,Win a Date with Tad Hamilton! (2004),10.0
89,117590,1.0,Horrible Bosses 2 (2014),10.0
88,1791,1.0,Twilight (1998),16.0
86,6265,1.0,Head of State (2003),15.0
81,32213,1.0,Cube Zero (2004),14.0
79,47970,1.0,"Last Kiss, The (2006)",17.0


###Line by line code

In [ ]:
rating_count = pd.DataFrame(ratings.groupby('movieId')['rating'].mean())
rating_count['rating_count'] = ratings.groupby('movieId')['rating'].count()
rating_count

In [ ]:
rating_count.drop(1, inplace=True)


In [ ]:
movie_corr_summary = correl_df1.join(rating_count['rating_count'])
movie_corr_summary

In [ ]:
top10 = movie_corr_summary[movie_corr_summary['rating_count']>=10].sort_values(0, ascending=False).head(10)
top10

##Personalised one
###Function

In [ ]:
def user_based_rec(userId, num_rec_items):
    
    rating_pivot = pd.pivot_table(ratings, values = 'rating', columns = 'movieId', index = 'userId') # create a pivot table
    users_items.fillna(0, inplace=True) # filling/replacing NaNs with 0
    user_similarities = pd.DataFrame(cosine_similarity(users_items), # compute cosine similarities
                                 columns=users_items.index, 
                                 index=users_items.index)
    user_Id = userId
    weights = (user_similarities.query("userId!=@user_Id")[user_Id] / sum(user_similarities.query("userId!=@user_Id")[user_Id])) # compute users' weights
    not_watched_movies = users_items.loc[users_items.index!=user_Id, users_items.loc[user_Id,:]==0] # select movies that the target user has not watched
    weighted_averages = pd.DataFrame(not_watched_movies.T.dot(weights), columns=["predicted_rating"]) # dot product between the not-watched-movies and the weights
    recommendations = weighted_averages.merge(movies, left_index=True, right_on="movieId")
    return recommendations.sort_values("predicted_rating", ascending=False).head(num_rec_items)


In [ ]:
user_based_rec(61,10)

###Line by line code

In [ ]:
#create a pivot table(this was already done above)
#rating_pivot = pd.pivot_table(ratings_1, values = 'rating', columns = 'movieId', index = 'userId')
users_items = rating_pivot.copy()
rating_pivot[18].sort_values() #we don't have 0 in our rating system, so we can use it as a filler for NaNs

In [ ]:
#filling/replacing NaNs with 0
users_items.fillna(0, inplace=True)
users_items.head()

In [ ]:
#Compute cosine similarities
from sklearn.metrics.pairwise import cosine_similarity

user_similarities = pd.DataFrame(cosine_similarity(users_items),
                                 columns=users_items.index, 
                                 index=users_items.index)
user_similarities.head()

In [ ]:
#Compute the weights
user_id = 1
weights = (user_similarities.query("userId!=@user_id")[user_id] / sum(user_similarities.query("userId!=@user_id")[user_id]))
weights.head(6)

In [ ]:
weights.sum()

In [ ]:
#Find items that the target user has not rated.
#Exclude the user, not to include him on the weights.
users_items.loc[user_id,:]==0

In [ ]:
# select itemss that the target user has not visited
not_visited_restaurants = users_items.loc[users_items.index!=user_id, users_items.loc[user_id,:]==0]
not_visited_restaurants.T

In [ ]:
# dot product between the not-visited-restaurants and the weights
weighted_averages = pd.DataFrame(not_visited_restaurants.T.dot(weights), columns=["predicted_rating"])
weighted_averages.head()

In [ ]:
recommendations = weighted_averages.merge(movies, left_index=True, right_on="movieId")
recommendations.sort_values("predicted_rating", ascending=False).head()